<a href="https://colab.research.google.com/github/ssumatoha/2048/blob/master/Dymov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [195]:
import torch
import torch.nn as nn
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn.functional as F
import torchvision as tv
from torch.utils.data import DataLoader, TensorDataset, Dataset
from tqdm.autonotebook import tqdm
from collections import Counter
import random
from sklearn.preprocessing import LabelEncoder

In [3]:
!pip install -q torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 kB 13.4 MB/s eta 0:00:00


In [4]:
from torchmetrics import Accuracy

In [ ]:
!pip install kaggle

In [5]:
from google.colab import files
files.upload()

Saving dataset.zip to dataset.zip


In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d moltean/fruits

 99% 1.27G/1.28G [00:09<00:00, 209MB/s]
100% 1.28G/1.28G [00:09<00:00, 141MB/s]


In [6]:
!unzip dataset.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_137_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_138_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_139_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_14_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_140_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_141_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_142_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_143_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_144_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_145_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_146_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_147_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax Red/r_148_100.jpg  
  inflating: dataset/Training/Cherry/Cherry Wax R

In [ ]:
!nvidia-smi

Fri Aug 25 08:35:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [196]:
class FruitDataset(Dataset):

    def __init__(self, image_paths, labels):
        super().__init__()
        self.image_paths = image_paths
        self.labels = labels

        self.transform = tv.transforms.Compose([
            tv.transforms.Resize((100, 100)),
            tv.transforms.ToTensor(),
            tv.transforms.RandomHorizontalFlip(),
            tv.transforms.RandomRotation(10),
        ])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        label = self.labels[index]

        image = Image.open(image_path)
        tensor_image = self.transform(image)
        # Дополнительные преобразования изображения, если нужно
        # Например, можно использовать torchvision.transforms для изменения размера или нормализации

        return tensor_image, label

In [197]:
def get_file_paths(root_dir):
    file_paths = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    return file_paths

In [198]:
def init_dataset(name: str):
  # Путь к директории с фотографиями
  dataset_path = f"/content/dataset/{name}"

  # Получение списка директорий в датасете
  directories = os.listdir(dataset_path)

  # Создание пустого массива для хранения пар элементов
  images = []
  labels = []

  # Проход по каждой директории и загрузка фотографий
  for directory in directories:
      directory_path = os.path.join(dataset_path, directory)
      image_files = os.listdir(directory_path)

      # Загрузка каждой фотографии и ее метки в массив
      for image_file in image_files:

          image_paths = get_file_paths(os.path.join(directory_path, image_file))
          label = directory

          # Добавление элементов в массивы
          for image_path in image_paths:

            images.append(image_path)
            labels.append(label)
          #images.append(image_path)
          #labels.append(label)

  return [images, labels]

In [167]:
images

NameError: ignored

In [199]:
train_x_paths, train_y = init_dataset("Training")
test_x_paths, test_y = init_dataset("Test")

'/content/dataset/Training/Pepper/Pepper Yellow/74_100.jpg'

In [200]:
# Узнаю количество уникальных значений
unique_values = set(train_y)
count_unique = len(unique_values)
count_unique

4

In [94]:
len(train_y)

13639

In [201]:
# Создаем объект класса LabelEncoder
label_encoder = LabelEncoder()

# Преобразуем категории в числовой формат
encoded_train_y = label_encoder.fit_transform(train_y)

encoded_test_y = label_encoder.transform(test_y)

In [202]:
def randing(images, labels):
  # Перемешиваю значения и метки
  data = list(zip(images, labels))
  random.shuffle(data)
  images, labels = zip(*data)
  return [images, labels]

In [203]:
# Перемешиваю значения
train_x_paths, encoded_train_y = randing(train_x_paths, encoded_train_y)
test_x_paths, encoded_test_y = randing(test_x_paths, encoded_test_y)

In [98]:
len(train_x_paths)

13639

In [204]:
# Инициализирую датасеты с тренировочной и обучающей выборками
train_dataset = FruitDataset(train_x_paths, encoded_train_y)
test_dataset = FruitDataset(test_x_paths, encoded_test_y)

In [226]:
train_dataset[0][0][0]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [205]:
BATCH = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH)

TypeError: ignored

In [17]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [18]:
DEVICE

'cuda'

In [212]:
class ConvNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.act = nn.LeakyReLU(0.2)
    self.maxpool = nn.MaxPool2d(2, 2)
    self.conv0 = nn.Conv2d(3, 32, 3, stride=1, padding=0)
    self.conv1 = nn.Conv2d(32, 32, 3, stride=1, padding=0)
    self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=0)
    self.conv3 = nn.Conv2d(64, 64, 3, stride=1, padding=0)
    self.conv4 = nn.Conv2d(64, 64, 3, stride=1, padding=0)

    self.adaptivepool = nn.AdaptiveAvgPool2d((1, 1))
    self.flatten = nn.Flatten()
    self.linear1 = nn.Linear(64, 10)
    self.linear2 = nn.Linear(10, 4)


  def forward(self, x):

    out = self.conv0(x)
    out = self.act(out)
    out = self.maxpool(out)

    out = self.conv1(out)
    out = self.act(out)
    out = self.maxpool(out)

    out = self.conv2(out)
    out = self.act(out)
    out = self.maxpool(out)

    out = self.conv3(out)
    out = self.act(out)
    out = self.maxpool(out)

    out = self.adaptivepool(out)
    out = self.flatten(out)
    out = self.linear1(out)
    out = self.act(out)
    out = self.linear2(out)

    return out

In [213]:
model = ConvNet()

In [208]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Создаем модель
model = ConvNet()
# Подсчитываем количество параметров в модели
num_params = count_parameters(model)
print(f"Количество параметров модели: {num_params}")

Количество параметров модели: 103190


In [214]:
# Определение функции потерь и оптимизатора
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.999))

In [215]:
accuracy = Accuracy(task="multiclass", num_classes=4)

In [194]:
for sample in train_loader:
  img = sample[0]
  label = sample[1]
  model(img)
  break

In [ ]:
epochs = 10

for epoch in range(epochs):
  loss_val = 0
  acc_val = 0
  for sample in (pbar := tqdm(train_loader)):
    img, label = sample[0], sample[1]
    optimizer.zero_grad()

    label = F.one_hot(label, 4).float()
    pred = model(img)
    loss = criterion(pred, label)

    loss.backward()
    loss_item = loss.item()
    loss_val += loss_item

    optimizer.step()

    acc_current = accuracy(pred, label)
    acc_val += acc_current.item()

  pbar.set_description(f'loss: {loss_item: .5f}\taccuracy: {acc_current: .3f}')
  print(loss_val/len(train_loader))
  print(acc_val/len(train_loader))

  0%|          | 0/426 [00:00<?, ?it/s]

0.3851904813287684
0.0


  0%|          | 0/426 [00:00<?, ?it/s]

0.24138656505186792
0.0


  0%|          | 0/426 [00:00<?, ?it/s]

0.11673021367685475
0.0


  0%|          | 0/426 [00:00<?, ?it/s]

In [1]:
for sample in (pbar := tqdm(train_loader)):
    img, label = sample[0], sample[1]
    print(label)
    print(img)
    break

NameError: ignored

# ниже запасной план

In [ ]:
class ConvNet(nn.Module):
  def __init__(self):
    super().__init__()

    self.act = nn.LeakyReLU(0.2)
    self.maxpool = nn.MaxPool2d(2, 2)
    self.conv0 = nn.Conv2d(3, 32, 3, stride=1, padding=0)
    self.conv1 = nn.Conv2d(32, 32, 3, stride=1, padding=0)
    self.conv2 = nn.Conv2d(32, 64, 3, stride=1, padding=0)
    self.conv3 = nn.Conv2d(64, 64, 3, stride=1, padding=0)
    self.conv4 = nn.Conv2d(64, 64, 3, stride=1, padding=0)

    self.adaptivepool = nn.AdaptiveAvgPool2d((1, 1))
    self.flatten = nn.Flatten()
    self.linear1 = nn.Linear()
    self.linear2 = nn.Linear()


  def forward(self, x):

    out = self.conv0(x)
    out = self.act(out)
    out = self.maxpool(out)

    out = self.conv1(out)
    out = self.act(out)
    out = self.maxpool(out)

    out = self.conv2(out)
    out = self.act(out)
    out = self.maxpool(out)

    out = self.conv3(out)
    out = self.act(out)
    out = self.maxpool(out)


    return out

In [ ]:
net = ConvNet()

In [ ]:
for sample in train_loader:
  img = sample[0]
  label = sample[1]
  net(img)
  break

torch.Size([64, 3, 100, 100])
torch.Size([64, 32, 49, 49])
torch.Size([64, 32, 23, 23])
torch.Size([64, 64, 10, 10])
torch.Size([64, 64, 4, 4])


In [ ]:
img.shape

torch.Size([64, 3, 100, 100])

In [ ]:
torch.cuda.empty_cache()
!nvidia-smi
torch.cuda.memory_allocated()

Wed Aug 23 15:31:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

0